# Seals On Ice: Mapping ice habitat for harbor seals in tidewater glacier fjords
## Quantifying ice change in Kenai Fjords National Park through monthly acquisitions using fixed wing aircraft and aerial photos.

#### _National Park Service, Alaska Regional Office, Scientist in Parks Program_

This Jupyter Notebook explains the background for a habitat monitoring project and explains the analytical workflow for imagery data. By using a shareable, annotatable format like a Jupyter Notebook, the steps in our remote sensing/GIS processing can be better understood and modified for similar projects by other researchers.

Graham Brady, Chad Hults, Deb Kurtz

## Background

Our project considers temporal change in habitat for harbor seals over the course of a pupping season. Seals rely on icebergs to haul out, seeking protection from predators and areas to bask _among other things_. Many tidewater glaciers in Alaska are  on developing a standalone method to map available ice habitat in three tidewater glacier fjord study areas over multiple fixed-wing aircraft acquisitions during the summer calving season of the glaciers. During the first year of acquisitions, sufficient data were gathered over McCarty Fjord, Northwestern Fjord, and Aialik Bay. This notebook includes processing steps that were applied to these fjords for May, July, and August 2023 imagery produced using a modified Structure-From-Motion photogrammetry workflow in Agisoft Metashape. Thematic maps of habitable ice were created using the remote sensing/GIS workflow in ArcGIS Pro and then were used to assess temporal change over the course of the summer in each study area. In the future, this workflow will be applied to additional acquisitions in the same study areas in 2024 and 2025. 

## Methods

### Acquisition

Air photo acquisition took place according to established methods at the Alaska Regional Office of the National Park service (cite). Using a Cessna 206H outfitted with underside camera ports and a G8 antenna, aerial surveys were flown along predetermined flightlines to take photos over the glacier, fjord, and nearby mountains with 80% overlap and 60% sidelap. An intervalometer attached to a Nikon D850 camera with 35mm lens allowed for photos to be taken with these coverage parameters. With a modified hot-shoe connector, camera shutters were linked to a Trimble Alloy GNSS receiver to trigger events during a 10Hz logging session at the same time a photo was taken.

### Structure-from-Motion Processing

Processing of raw air photos includes two main stages: production of a three-band orthoimage and thematic mapping of that orthoimage. Air photos were linked to a survey grade point in space from which they were taken. A table of these file names and reference points were added to Agisoft Metashape in the Alaksa State Plane Zone 4 coordinate system using the North American Datum 1983 epoch 2011. 

Photos were aligned using the software's standard tool which searches for matches in overlapping photographs (tie points). Because water and floating ice are not stationary during the period of photo acquisition, tie points of these types were deleted to increase the accuracy of the camera calibration; by optimizing camera variables to the subset of tie points that represent true, stationary matches, the resultant DEM and orthomosaic are thus more accurate over land.

However, because there are minimal areas over water to find matches between photos, it was necessary to hydroflatten the water bodies and icebergs. Production of the orthophoto used the "mosaic" method within Agisoft Metashape to blend boundaries between photos for the final stitched product.

_include information about photo duplicates here_

### Thematic Mapping of Icebergs

Several methods of image classification and GIS processing were attempted before developing a finalized workflow. While individual images could be robustly mapped using a particular approach, few of these approaches were successful for multiple fjords or months of acquisitions. However, leveraging the positive aspects of prior attempts proved helpful for develping a solution that worked for many acquisitions. Because of changing lighting conditions, variability in ice density, and differences in surface water appearance, it was necessary to develop a workflow that produces accurate thematic maps to compare available ice area with minimal alteration or input from an analyst beyond project setup.

The Jupyter Notebook format incorporates blocks of codes and annotations to document processing, show results, and provide a way to share workflows wither others. While developing the image classification workflow for this project, stages were written in Python code format and the final version is included below.

Most of the geoprocessing for this workflow utilizes ESRI's ArcPy library of classes and methods, which blend standard python techniques and syntax with a simple set of tools that resemble the organization of ArcMap and ArcGIS Pro geoprocessing tools. With an understanding of the way these tools work and some creativity, the Jupyter Notebook is an optimal environment through which to develop and improve an analytical workflow, particularly in a repetitive way. With the ability to quickly cycle through parameters we were able to determine the optimal settings and combinations of variables with which to produce the best result across acquisitions



#### Importing libraries and setting filepaths to directories for reading and writing of data

Python is powerful because of the ability to load open source libraries created by other users and groups that are tailored to particular uses. Ranging from data analysis, computer vision, statistics, and graphing libraries, research developed using python often combines multiple libraries to create simple, efficient solutions.

The SealsOnIce workflow relies on access to ESRI licensing and was developed using ArcGIS Pro. Therefore, the main workhorse of the processing steps is `arcpy`, an ESRI library that transitions geoprocessing tools to a scripting format to analyze and map data without using the graphical interface of ArcGIS Pro. Because NPS researchers have enterprise access to this software, we chose to use this service for our workflow because it is an easy, reproducable way to share our research approach with the ability to also carry out the same steps without a coding background.

To setup the `arcpy` environment, it is necessary to assign a working directory, project, and map. Using the main project folder and its subdirectories, we can do that.

In [3]:
#import arcpy
import os
import numpy as np
import pandas as pd

The `os` library contains methods to interact with the file explorer and manipulate paths while retaining proper formatting. `os.path.join()` is an example of a method, implemented above to access subdirectories of the main project folder `proj_folder =  r'E:\2023_07_13_KEFJ_TidewaterGlaciers'`.

Another useful method is `os.walk()`, which will recursively return the contents of a given directory. This allows for processing within all subdirectories of data. In our case we can use this to apply the same workflow across all acquisitions if we structure our data storage as follows:

- Analysis Folder
    - 2023_05_18_Acquisitions
        - McCarty
        - Northwestern
        - Aialik
    - 2023_07_13_Acquisitions
        - McCarty
        - Northwestern
        - Aialik
    - 2023_08_18_Acquisitions
        - McCarty
        - Northwestern
        - Aialik
        
We can run `os.listdirs(r'E:\KEFJ_TidewaterGlaciers_Analysis')` to return objects for each aquisition folder, and proceed to apply the same steps to those folder objects. To make sure we only get the directories and not additional lingering files, we must subset the result using a conditional. Additionally we subset away the main __Arc__ folder, since that will not contain any acquisition-specific data.

In [32]:
main_folder = r'F:\KEFJ_TidewaterGlaciers_Analysis'
year_folders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f)) and f != 'Arc'] 

In [33]:
year_folders

['August_2023', 'July_2023', 'May_2023']

Using a List Comprehension technique, the year folders within the main folder are subset for just directories (dirs) not named 'Arc'. From this point, we look for additional dirs within each year folder, check that they are dirs not files, and concatenate them to a flattened list of acquisitions.

In [40]:
acquisitions = []
for year_folder in year_folders:
    acquisitions.append([os.path.join(main_folder, year_folder, f) for f in os.listdir(os.path.join(main_folder, year_folder)) if os.path.isdir(os.path.join(main_folder, year_folder, f))])
    study_areas = [f for f in os.listdir(os.path.join(main_folder, year_folder)) if os.path.isdir(os.path.join(main_folder, year_folder, f))]
acquisitions = [item for row in acquisitions for item in row] #flatten the list of lists from above loop
print('The following acquisitions have been found in the main folder', main_folder, ':')
acquisitions

The following acquisitions have been found in the main folder F:\KEFJ_TidewaterGlaciers_Analysis :


['F:\\KEFJ_TidewaterGlaciers_Analysis\\July_2023\\Aialik',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\July_2023\\McCarty',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\July_2023\\Northwestern',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\May_2023\\Aialik',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\May_2023\\McCarty',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\May_2023\\Northwestern']

At this stage of processing, the workflow will be demonstrated on only May and July 2023.

In [35]:
year_folders = year_folders[1:]
year_folders

['July_2023', 'May_2023']

In [36]:
acquisitions = []
for year_folder in year_folders:
    acquisitions.append([os.path.join(main_folder, year_folder, f) for f in os.listdir(os.path.join(main_folder, year_folder)) if os.path.isdir(os.path.join(main_folder, year_folder, f))])                      
acquisitions = [item for row in acquisitions for item in row] #flatten the list of lists from above loop
print('The following acquisitions have been found in the main folder', main_folder, ':')
acquisitions

The following acquisitions have been found in the main folder F:\KEFJ_TidewaterGlaciers_Analysis :


['F:\\KEFJ_TidewaterGlaciers_Analysis\\July_2023\\Aialik',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\July_2023\\McCarty',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\July_2023\\Northwestern',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\May_2023\\Aialik',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\May_2023\\McCarty',
 'F:\\KEFJ_TidewaterGlaciers_Analysis\\May_2023\\Northwestern']

In [41]:
study_areas

['Aialik', 'McCarty', 'Northwestern']

### For testing purposes: let's just use McCarty July 2023.
____________________________________________

### Stage 1: Image Segmentation

In [42]:
acquisition = acquisitions[1]
acquisition
year_folder = 'July_2023'
study_area = study_areas[1]

In [44]:
layer_prefix = year_folder + '_' + study_area + '_'

'July_2023_McCarty_'

In [47]:
map_query = study_area + ' BandStack Classification'
map_query

'McCarty BandStack Classification'

In [ ]:
#proj_folder = acquisition
proj_folder =  r'E:\2023_07_13_KEFJ_TidewaterGlaciers'
data_path = os.path.join(proj_folder, 'Image_RGB') #folder containing RGB image
dst_path = os.path.join(proj_folder, 'Classification\Layers')
shapes_path = os.path.join(proj_folder, 'Classification\Shapes')
chart_dst = os.path.join(proj_folder, 'Classification\Charts')
project_gdb = os.path.join(proj_folder, 'Arc','2023_07_13_KEFJ_TidewaterGlaciers.gdb')

rgb_filename = '2023_07_13_KEFJ_McCarty_ImageRGB.tif' 

#layer_prefix = year_folder + '_' + study_area + '_'

#### Assigning variable names for the current acquisition

Because the same workflow will be applied to each acquisition's folder, it is necessary to create a naming convention that adds descriptors after a general stem. Within the folder, each RGB image is located in the same subdirectory and the outputs are to be written in the same subdirectory. For example, the RGB image will be gathered by searching for a .tif file whose name ends in _Image_RGB.tif_ in the __Image_RGB__ folder and will calculate and write a bandstack with name _Bandstack_Value_NDWI_BNDVI.tif_ in the __Layers__ folder. These file names can be distinguised by adding _KEFJ_Northwestern_July_2023_ as a prefix. We can retrieve the proper prefix from the original names of the acquisition folders during our setup. Additionally, steps later on in can access the same types of intermediary rasters to continue the analysis without having to alter much of the code during setup.


In [ ]:
arcpy.env.workspace = data_path
arcpy.env.overwriteOutput = True
aprx = arcpy.mp.ArcGISProject("Current")
map_query = study_area + ' BandStack Classification'
m = aprx.listMaps(map_query)[0]

Once the read and write paths for data for the particular acquisition to be processared are defined in `arcpy`, we start to work within an environment similar to a Map in ArcGISPro.

In [ ]:
mask = m.listLayers('2023_07_13_KEFJ_McCarty_IceArea')[0] #search for matching layer name in the map
rgb = arcpy.sa.Raster(rgb_filename) #create a raster object from the filename loaded into the map
rgb_clip = arcpy.ia.Clip(rgb, aoi = mask.name) #clip the raster
rgb_clip.save(os.path.join(dst_path, layerPrefix+'IceAreaClip.tif')) #save the clipped raster to the destination path with the proper name

#### Producing relevant raster layers from input data

After importing the data, 

In [ ]:
RGB = rgb_clip

b1, b2, b3 = RGB.getRasterBands()
NullBinaryRaster = arcpy.sa.RasterCalculator([b1,b2,b3],['b1','b2','b3'],'(b1 == 0) & (b2 == 0) & (b3 == 0)')
NullBinaryRaster.save('nullBinaryRaster.tif')
arcpy.CompositeBands_management(';'.join([str(NullBinaryRaster.catalogPath)]*RGB.bandCount),
                                "nullBandRasterStack.tif")
RGB2 = arcpy.sa.SetNull("nullBandRasterStack.tif", RGB, 'Value = 1')

#RGB = arcpy.sa.Raster('RGB2') bug here,
RGB = arcpy.sa.Raster(RGB2.name)

Band arithmetic is a common tool derive additional rasters in remote sensing data analysis. Also known as map algebra or band math in other software, the result of this two is a raster whose values represent the result of the expression for each cell in one or more input rasters. With rasters of different shape, cell size adjustments are necessary. However, in this context, we are operating on multiple bands of the same input raster, _Image_RGB_. In other circumstances, we might combine data from several sources to calculate temperature from radiance and emissivity. 

For our project, we developed several normalized band indices to accentuate differences in land cover and subdue differences in lighting conditions. These indices were modified from established indices featured in studies using calibrated sensors. For example, satellite bands calibrated for reflectance produce accurate and precise measurements for vegetation productivity with the Normalized Difference Vegetation Index (NDVI). While our cameras are not calibrated, the object based segmentation method in our workflow weights local differences in pixel values, supporting our use of these modified indices.

In [ ]:
ndwi = arcpy.sa.BandArithmetic(RGB, "(B2-B1)/(B2+B1)", 0)
ndwi.save(os.path.join(dst_path, layerPrefix+'NDWI.tif'))

bndvi = arcpy.sa.BandArithmetic(RGB, "(B1-B3)/(B1+B3)", 0)
bndvi.save(os.path.join(dst_path, layerPrefix+'BNDVI.tif'))

endvi = arcpy.sa.BandArithmetic(RGB, "((B1+B2)-(2*B3))/((B1+B2)+(2*B3))", 0)
endvi.save(os.path.join(dst_path, layerPrefix+'ENDVI.tif'))

After calculating the rasters from the RGB image, we add them to the map and assign each as a raster object.

In [ ]:
m.addDataFromPath(os.path.join(dst_path, layerPrefix+'BNDVI.tif'))
m.addDataFromPath(os.path.join(dst_path, layerPrefix+'ENDVI.tif'))
m.addDataFromPath(os.path.join(dst_path, layerPrefix+'NDWI.tif'))

ndwi = arcpy.sa.Raster(m.listLayers('*NDWI*')[0].dataSource)
bndvi = arcpy.sa.Raster(m.listLayers('*BNDVI*')[0].dataSource)
endvi = arcpy.sa.Raster(m.listLayers('*ENDVI*')[0].dataSource)

Jamie Womble's research group conducted a similar study in Glacier Bay National Park and Preserve using imagery gathered by UAS. Their analysis converted the imagery from RGB colorspace to hue, saturation, and value (HSV). The value band of this colorspace corresponds to the brightness level for pixels. Because ice is significantly brighter than surrounding water, the value raster lends useful for a quick thresholding step by which to discriminate ice and water. 

While in some attempts a single threshold number of the value raster led to a successful thematic map, our team found it difficult to develop an automated method of calculating an image-specific threshold number for the value raster using sampled training points. However, because of the usefulness of this raster layer at highlighting the differences between ice and water, we included these value rasters in the bandstack that is fed into the segmentation tool.

In [ ]:
out_hsv_3bands = arcpy.sa.ColorspaceConversion(RGB, "rgb_to_hsv") #convert RGB image to HSV
out_hsv_3bands.save(os.path.join(dst_path, layerPrefix+'hsv_3bands.tif')) #save HSV image

m.addDataFromPath(os.path.join(dst_path, layerPrefix+'hsv_3bands.tif')) #add HSV image to map

value = arcpy.sa.Raster(layerPrefix+'hsv_3bands.tif') #create raster object for hsv raster
h, s, v = value.getRasterBands() #enumerate the three bands of the image, with v (value) being the band of interest

To leverage the best qualities of each derived band, we create a new three band composite raster for the Value, BNDVI, and ENDVI rasters.

In [ ]:
arcpy.management.CompositeBands([v, bndvi, endvi], os.path.join(dst_path, layerPrefix+'Val_Bndvi_Endvi_Composite.tif')) #create a bandstack of the three bands
bs_clip = arcpy.ia.Clip(arcpy.sa.Raster(layerPrefix+'Val_Bndvi_Endvi_Composite.tif'), aoi = mask.name) #clip the bandstack once more to remove and NoData pixels around the edges
bs_clip.save(os.path.join(dst_path, layerPrefix+'Val_Bndvi_Endvi_Composite_IceClip.tif')) #save the clipped bandstack 

After adding the bandstack clipped to the area of interest and creating a raster object, set the parameters for the segmentation algorithm and run the tool to produce a shapefile of likely ice polygons. Some clean up will be needed, but the preliminary segmentation result includes polygons of all sizes that were created using the segment mean shift method.

In [ ]:
m.addDataFromPath(os.path.join(dst_path, layerPrefix+'Val_Bndvi_Endvi_Composite_IceClip.tif')) #add the clipped raster to the map

r = arcpy.sa.Raster(layerPrefix+'Val_Bndvi_Endvi_Composite_IceClip.tif') #assign a raster object

#set the segmentation settings
spectral = 10
spatial = 10
minSize = 25

segmented = arcpy.ia.SegmentMeanShift(r, spectral, spatial, minSize, '', -1)
segmented.save(os.path.join(dst_path, layerPrefix+'BandStack_ValueBNDVIENDVI_Segmentation' + str(spectral) + '_' + str(spatial) +'_'+ str(minSize) + '.tif'))

arcpy.conversion.RasterToPolygon(segmented, os.path.join(shapes_path, layerPrefix+'BandStack_ValueBNDVIENDVI_Segmentation' + str(spectral) + '_' + str(spatial) +'_'+ str(minSize) +'_polygonsIceClip.shp'), "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

### Stage 2: Polygon Refinement

Now that the preliminary thematic map has been produced and converted to polygon format, polygons need to be attributed and  cleaned. Size thresholds must be applied to subset the result to include ice habitable by harbor seals. Jaimie Womble's group notes that this size threshold is greater than 1.6 square meters.


To refine the polygons, we need to also remove any suspected water polygons using size and mean brightness (value) thresholds.

#### Running the workflow

Once again, we can use the directory organization to our advantage by accessing the name of the acquisition folder and year that we are currently processing.

In [ ]:
#these should all be the same from the first stage
aprx = arcpy.mp.ArcGISProject("CURRENT") #set ArcGISPro project

segmentation_result_path = os.path.join(shapes_path, layerPrefix+'BandStack_ValueBNDVIENDVI_Segmentation' + str(spectral) + '_' + str(spatial) +'_'+ str(minSize) +'_polygonsIceClip.shp') #assign path to polygons
image_rgb_path = os.path.join(data_path, rgb_filename) #assign RGB image path
value_raster_path = os.path.join(dst_path, layerPrefix+'hsv_3bands.tif\Band_3') #assign value raster path

A new map is created and relevant data added. We need to retrive the names of files, folders, and databases from the segmentation phase of the workflow.

In [ ]:
refinement_map = aprx.createMap(study_area + " Ice Classification Refinement") #create new map
img_rgb = refinement_map.addDataFromPath(image_rgb_path) #add RGB image for basemap
img_rgb.name = 'Image_RGB' #change display name
val = refinement_map.addDataFromPath(value_raster_path) #add value raster for zonal statistics and refinement
val.name = 'Value Raster' #change display name
seg_result = refinement_map.addDataFromPath(segmentation_result_path) #add segmented polygons
seg_result.name = 'Segmentation Result' #change display name

In [ ]:
#segmentation_result_path = r'E:\2023_07_13_KEFJ_TidewaterGlaciers\Image_RGB\2023_07_13_KEFJ_McCarty_V2_BandStack_ValueBNDVIENDVI_Segmentation10_10_25_polygonsIceClip.shp'

In [ ]:
segmentation_result = seg_result.name #assign current name for reference to the polygon layer in the catalog
value_raster = val.name #assign current name for reference to the value layer in the catalog

#####CHANGE THESE
area_subset = year_folder + study_area + '_Refined_AreaSubset'
zonal_stats_table = year_folder + study_area + '_ValueZonalStats_QC_AreaSubset'
near_table = year_folder + study_area + '_QC_AreaSub_NearTable'

#gdb = r"E:\2023_07_13_KEFJ_TidewaterGlaciers\Arc\2023_07_13_KEFJ_TidewaterGlaciers.gdb"


#####CHANGE THESE

Calculate geometry attributes in State Plane Zone 4 with NAD1983 Epoch 2011.

In [ ]:
with arcpy.EnvManager(scratchWorkspace=r"E:\2023_07_13_KEFJ_TidewaterGlaciers\Arc\2023_07_13_KEFJ_TidewaterGlaciers.gdb", workspace=r"E:\2023_07_13_KEFJ_TidewaterGlaciers\Arc\2023_07_13_KEFJ_TidewaterGlaciers.gdb"):
    arcpy.management.CalculateGeometryAttributes(
        in_features=segmentation_result,
        geometry_property="area_sqm AREA",
        length_unit="",
        area_unit="SQUARE_METERS",
        coordinate_system='PROJCS["NAD_1983_2011_StatePlane_Alaska_4_FIPS_5004",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-150.0],PARAMETER["Scale_Factor",0.9999],PARAMETER["Latitude_Of_Origin",54.0],UNIT["Meter",1.0]],VERTCS["unknown",VDATUM["unknown"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]',
        coordinate_format="SAME_AS_INPUT"
    )

After calculating geometry attributes for each polygon, the features are subset and exported to include polygons larger than the minimum size threshold and smaller than an extemely large threshold. Since the segmentation algorithm does not assign any designation as ice, water, or other, it is assumed that the largest polygon represents the majority of the water in the study area. From tests, deleting this largest polygon results in a subset of very likely ice polygons with further clean up of small areas of submerged ice and water pockets adjacent to ice.

In [ ]:
arcpy.conversion.FeatureClassToFeatureClass(
    in_features= segmentation_result,
    out_path=gdb,
    out_name= area_subset,
    where_clause="area_sqm >= 1.6 And area_sqm < 3000",
    config_keyword=""
)
area_sub = refinement_map.listLayers(area_subset)[0] #assign variable to subset lyr

Symbology is also applied for the subset polygons and the original result for ease of identification should this workbook be run with an ArcGIS Pro window open at the same time.

In [ ]:
sym = seg_result.symbology
sym.renderer.symbol.applySymbolFromGallery("Black Outline (2 pts)")
sym.renderer.symbol.outlineColor = {'RGB' : [255, 0, 0, 255]} #rgb alpha
seg_result.symbology = sym #apply red to original layer
sym.renderer.symbol.outlineColor = {'RGB' : [0, 255, 0, 255]}
area_sub.symbology = sym #apply green to subset layer

The remainder of polygons must be cleaned according to their pixel values. By calculating zonal statistics for the polygons and considering the distribution of those values, a threshold can be determined to coarsely refine the polygons and remove likely water polygons. Results of the zonal statistics tool are joined to the polygon layer and a histogram is created.

In [ ]:
arcpy.ia.ZonalStatisticsAsTable( #produce zonal stats for the mean value for each polygon
    in_zone_data=area_subset,
    zone_field="Id",
    in_value_raster= value_raster,
    out_table=os.path.join(gdb, zonal_stats_table),
    ignore_nodata="DATA",
    statistics_type="ALL",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[90,75],
    percentile_interpolation_type="LINEAR"
)

arcpy.management.JoinField( #join zonal stats to area refined table
    in_data=area_subset,
    in_field="Id",
    join_table=zonal_stats_table,
    join_field="Id",
    fields=None
)

In [ ]:
area_sub = refinement_map.listLayers(area_subset)[0]
mean_val_chart = arcpy.charts.Histogram("MEAN", binCount=40)
mean_val_chart.dataSource = area_sub
mean_val_chart.title = "Distribution of Mean Pixel Value"
mean_val_chart.addToLayer(area_sub)
mean_val_chart.exportToSVG("mean_val_hist.svg", width=1000, height=800)
#take a look at the chart in the catalog

__Display the chart here__

A final, fine adjustment to the ice polygons is required to remove polygons that were mapped due to their proximity to true ice polygons. Using a near table that searches for touching polygons, a similar thresholding step is applied.

In [ ]:
arcpy.analysis.GenerateNearTable(
    in_features=area_subset,
    near_features=area_subset,
    out_table=os.path.join(gdb, near_table),
    search_radius="0 Meters", #adjacent, touching polygons
    location="NO_LOCATION",
    angle="NO_ANGLE",
    closest="ALL",
    closest_count=0,
    method="PLANAR"
)

# JOIN NEAR TABLE RESULTS TO AREA SUBSET
arcpy.management.JoinField(
    in_data=near_table,
    in_field="IN_FID",
    join_table=area_subset,
    join_field="OBJECTID",
    fields="MEAN",
)
arcpy.management.JoinField( #adds "MEAN_1 to the table, which is the mean_val of the NEAR polygon."
    in_data=near_table,
    in_field="NEAR_FID",
    join_table=area_subset,
    join_field="OBJECTID",
    fields="MEAN"
)
#change names of fields for user ease
arcpy.management.AlterField(near_table,
                            "MEAN",
                            "origin_mean",
                            "origin_mean")
arcpy.management.AlterField(near_table,
                            "MEAN_1",
                            "near_mean",
                            "near_mean")

After joining and renaming the fields from the near table to the polygon layer, a new metric is required to compare the values of two adjacent polygons. From tests, this normalized difference variant seems to help distinguish polygons that are markedly lower in mean brightness from their neighbor. Subsetting polygons with a positive value for this comparison metric fosuses on polygons lower then their neighbor.

In [ ]:
arcpy.management.CalculateField(
    in_table=near_table,
    field="difference",
    expression="(!origin_mean!-!near_mean!)/(!origin_mean!+!near_mean!)",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)
arcpy.management.SelectLayerByAttribute( #selects polygons where the adjacent polygon is lower.
    in_layer_or_view=near_table,
    selection_type="NEW_SELECTION",
    where_clause="difference > 0",
    invert_where_clause=None
)
arcpy.management.JoinField(
    in_data=area_sub,
    in_field="OBJECTID",
    join_table=near_table,
    join_field="NEAR_FID",
    fields=None
)

Producing an additional histogram  of this difference metric helps to determine the threshold value to carryout the last stage of data refinement based on pixel values before producing final histograms for iceberg area distribution for the acquisition.

In [ ]:
val_dif_chart = arcpy.charts.Histogram("difference", binCount=20)
val_dif_chart.dataSource = area_sub
val_dif_chart.title = "Distribution of Mean Pixel Value Difference"
val_dif_chart.addToLayer(area_sub)

__export chart svg and include inline here__

After using the histograms to determine the coarse and fine threshold values to subset the polygons based on mean pixel brightness values, we apply those thresholds in an export step and resymbolize.

In [ ]:
#use descriptive statistics to get values
arcpy.SelectLayerByAttribute_management(area_subset, "CLEAR_SELECTION")
arcpy.conversion.FeatureClassToFeatureClass(
    in_features=area_subset,
    out_path=gdb,
    out_name=area_subset + '_final_refined',
    where_clause="MEAN >= 3000 Or difference < .05", #input stats here for thresholds
    config_keyword=""
)
sym = area_sub.symbology
sym.renderer.symbol.applySymbolFromGallery("Black Outline (2 pts)")
sym.renderer.symbol.outlineColor = {'RGB' : [255, 255, 0, 255]} #rgbalpha
area_sub.symbology = sym #apply red to original layer



#### 

####

####

#### 